Flash Photolysis part to decide the concentration of Bacteriorhodopsin

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


file_path = '/Users/melisaaykul/Desktop/flash photolysis/steady-state/BR no Buffer_135805.txt'

data = pd.read_csv(file_path, sep='\t', header=2) 
data.columns = ['Wavelength(nm)', 'Absorbance(A.U)']

# Plotting part
plt.figure(figsize=(10, 5))
plt.plot(data['Wavelength(nm)'], data['Absorbance(A.U)'], linestyle='-')
plt.xlim(250, 700)  
# Adding a vertical line at x = 565 nm
plt.axvline(x=565, color='red', linestyle='--', linewidth=1.5, label='Wavelength 565 nm')
plt.legend()
plt.xlabel('Wavelength(nm)')
plt.ylabel('Absorbance(A.U)')
plt.grid(True)
plt.show()

Flash Photolysis part to compute pH of BR with pyranine and without pyranine cases.

In [ ]:

file_path = '/Users/melisaaykul/Desktop/determination2.txt'


data = pd.read_csv(file_path, sep='\t', header=2)  # Adjust sep='\t' if your data is not tab-separated
data.columns = ['Wavelength(nm)', 'Absorbance(A.U)']


plt.figure(figsize=(10, 5))
plt.plot(data['Wavelength(nm)'], data['Absorbance(A.U)'], linestyle='-')
plt.xlim(250, 700)  # Adjust these values if necessary to exactly match your desired range
# Adding a vertical line at x = 404 nm
plt.axvline(x=404, color='red', linestyle='--', linewidth=1.5, label='Wavelength 404 nm')
# Adding a vertical line at x = 454 nm
plt.axvline(x=454, color='green', linestyle='--', linewidth=1.5, label='Wavelength 454 nm')
# Adding a horizontal line at y = 454 nm
plt.axhline(y=0.5214707, color='black', linestyle='--', linewidth=1.5, label='A$_{404}$=0.521')
# Adding a horizontal line at y = 454 nm
plt.axhline(y=0.78643675, color='orange', linestyle='--', linewidth=1.5, label='A$_{454}$=0.786')

plt.legend()
plt.xlabel('Wavelength(nm)')
plt.ylabel('Absorbance(A.U)')
plt.grid(True)
plt.show()

Time Spectras at different wavelengths and their Gaussian fits. I have added one example.

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

file_path = '/Users/melisaaykul/Desktop/BR_550nm_630nm_cut.txt'
data = pd.read_csv(file_path, delimiter=',', header=None)
time = data.iloc[:, 0]
absorbance = data.iloc[:, 9]

def exp_growth(x, a, b, c):
    return a * np.exp(-b * x) + c

def exp_decay(x, a, b, c):
    return a * np.exp(b * x) + c

decay_mask = (time > 1e-6) & (time < 3e-4)
growth_mask = (time > 3e-4) & (time < 9e-3)


print(f"Growth data points: {np.sum(growth_mask)}")
print(f"Decay data points: {np.sum(decay_mask)}")

initial_decay_params = [-0.00220701, 1e3, 2] 
popt_growth, pcov_growth = curve_fit(exp_growth, time[growth_mask], absorbance[growth_mask], p0=initial_growth_params, maxfev=10000)

initial_growth_params = [0.0143673, 1e3, 0] 
popt_decay, pcov_decay = curve_fit(exp_decay, time[decay_mask], absorbance[decay_mask], p0=initial_decay_params, maxfev=10000)

fitted_growth = exp_growth(time[growth_mask], *popt_growth)
fitted_decay = exp_decay(time[decay_mask], *popt_decay)

plt.figure(figsize=(10, 6))
plt.plot(time, absorbance, 'b-', label='Data')
plt.plot(time[growth_mask], fitted_growth, 'g--', label='Growth fit')
plt.plot(time[decay_mask], fitted_decay, 'r--', label='Decay fit')
plt.xscale('log')
plt.xlabel('Time(s)')
plt.ylabel('Absorbance(A.U)')
plt.title('Time vs Absorbance with Exponential Fits at 550 nm')
plt.legend()
plt.grid(True)
plt.show()

print("Growth fit parameters:", popt_growth)
print("Decay fit parameters:", popt_decay)
print(f"Growth equation: A(t) = {popt_growth[0]:.5f} * exp({popt_growth[1]:.5e} * t) + {popt_growth[2]:.5f}")
print(f"Decay equation: A(t) = {popt_decay[0]:.5f} * exp(-{popt_decay[1]:.5e} * t) + {popt_decay[2]:.5f}")

Time Resolvef FTIR part - Absorbance Heat Map

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize


file_path = '/Users/melisaaykul/Desktop/step_scan1.csv'
data = pd.read_csv(file_path, delimiter=';')

wavenumbers = data.columns[1:].astype(float)
time = data.iloc[:, 0].astype(float)
absorbance_values = data.iloc[:, 1:].replace(',', '.', regex=True).astype(float).values

# Definition of the custom normalization for absorbance axis
norm = Normalize(vmin=-0.0012, vmax=0.0018)

# Heatmap
fig, ax = plt.subplots(figsize=(12, 6))

# Display the heatmap
cax = ax.imshow(absorbance_values, aspect='auto', cmap='plasma', norm=norm, origin='lower')

ax.set_xlabel(r'Wavenumber $\nu$ (cm$^{-1}$)', fontsize=20)
ax.set_ylabel('Time (s)', fontsize=15)
ax.set_title('Absorbance Heatmap', fontsize=18)

# Manually set y-ticks to match the required values
y_ticks = np.logspace(-7, 0, 8)  # From 10^-7 to 10^0
y_tick_labels = [r'$10^{' + f'{int(np.log10(t))}' + r'}$' for t in y_ticks]
y_indices = np.interp(np.log10(y_ticks), np.log10(time), np.arange(len(time)))

ax.set_yticks(y_indices)
ax.set_yticklabels(y_tick_labels, fontsize=10)

# Manually set x-ticks to match the required range
x_ticks = np.linspace(0, len(wavenumbers)-1, len(np.arange(1800, 700, -100)))
x_labels = np.arange(1800, 700, -100)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_labels, rotation=45, fontsize=10)

cbar = fig.colorbar(cax, ax=ax, orientation='vertical', ticks=np.arange(-0.0012, 0.0019, 0.0003))
cbar.set_label('Absorbance')

plt.show()

3D Map of Time Resolved FTIR

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


file_path = '/Users/melisaaykul/Desktop/3D_data.csv'
data = pd.read_csv(file_path, delimiter=';')

# Extraction of time (first column) and wavenumbers (first row, excluding the first element)
time = data.iloc[1:, 0].astype(float).values
wavenumbers = data.columns[1:].astype(float)

# Application of logarithmic scaling to the time values
log_time = np.log10(time)

# Extraction of absorbance values (excluding the first row and first column)
absorbance_values = data.iloc[1:, 1:].replace(',', '.', regex=True).astype(float).values

# Meshgrid for time and wavenumbers
T, W = np.meshgrid(log_time, wavenumbers)

# 3D plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(T, W, absorbance_values.T, cmap='plasma', edgecolor='none')


ax.set_xlabel('Time [s]')
ax.set_ylabel('Wavenumber [cm$^{-1}$]')
ax.set_zlabel('Absorbance')
ax.set_title('3D Absorbance Surface Plot')

# Set the time axis limits from 10^-8 to 10^0 and customize ticks
ax.set_xlim(np.log10(1e-8), np.log10(1e0))
ax.set_xticks(np.log10([1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0]))
ax.set_xticklabels([r'$10^{-8}$', r'$10^{-7}$', r'$10^{-6}$', r'$10^{-5}$', r'$10^{-4}$', r'$10^{-3}$', r'$10^{-2}$', r'$10^{-1}$', r'$10^{0}$'])

# Set z-axis limits
ax.set_zlim(-0.0015, 0.0015)


fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, label='Absorbance')

plt.show()
